<center><h1  style="color:white; background-color:#000000; border-radius: 0px; padding:25px;"> Theoretical study of SMOTE </h1></center>

This notebook reproduces the numerical experiments.

<ins>Experimented Datasets: </ins>
- [Phoneme](#Phoneme)
- Pima
- Abalone
- Haberman
- Yeast
- Wine
- Vehicle
- Bresat cancer Wisconsin
- Ionosphere
- CreditCard
- MagicTel
- California
- House_16H

ALl the sections of this notebook are indepent. user should run the following two introduction code chunk. Then one can choose a given data set section and run exclusivey the chunks code of the selected section. Fot the subsampled data sets version, all the seed and are given and the subsampling are each time included in the given section. However, the protocol is always pre-filed with the smalles (most undersampled) version of the dat set. On can try other subsampled data sets by changing the protocol data sets used.

In [ ]:
import os
import sys
sys.path.insert(1, os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from pathlib import Path


#from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, roc_auc_score
from imblearn.under_sampling import RandomUnderSampler,NearMiss
from imblearn.over_sampling import RandomOverSampler, SMOTE, BorderlineSMOTE


from oversampling_strategies.oversampling_strategies import MGS, NoSampling, CVSmoteModel,CtganSampler,ForesDiffSampler

from validation.classif_experiments import run_eval, subsample_to_ratio_indices,read_subsampling_indices, compute_metrics_several_protocols

In [ ]:
from imblearn.ensemble import BalancedBaggingClassifier

In [ ]:
#import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier

In [ ]:
output_dir_path_lgbm =  "../saved_experiments/rf_not_tuned" ## Fill it
Path(output_dir_path_lgbm).mkdir(parents=True, exist_ok=True)
output_dir_path_lgbm

In [ ]:
output_dir_path_lgbm

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;" name='Phoneme'> Phoneme</h1></center>

## <a name='Phoneme'></a>

## Run protocol :

In [ ]:
from data.data import load_phoneme_data
X_phoneme, y_phoneme = load_phoneme_data()

In [ ]:
depth_max = None

In [ ]:
smote_bagging = BalancedBaggingClassifier(sampler=SMOTE(),random_state=0)
mgs_bagging = BalancedBaggingClassifier(sampler=MGS(K=6,random_state=0),random_state=0)
#smote_boost = SMOTEBoostClassifier(random_state=0)

In [ ]:
lgb_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5,max_depth=depth_max)
balanced_lgb_model =  RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5,max_depth=depth_max)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=0)
CVSmotelgb = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5,max_depth=depth_max),
    list_k_max=15,list_k_step=1)

In [ ]:
output_dir_path =  "/home/abdoulaye_sakho/S1/th_smote/smote_strategies_study/saved_experiments_2025_v2/not_tuned/res_phoneme/original" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=6,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_phoneme,y=y_phoneme,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

In [ ]:
output_dir_path =  "/home/abdoulaye_sakho/S1/th_smote/smote_strategies_study/saved_experiments_2025_v2/not_tuned/res_phoneme/subsample_to_20/" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)
X_phoneme_20,y_phoneme_20 = read_subsampling_indices(X=X_phoneme,y=y_phoneme, dir_subsampling=output_dir_path_lgbm,
                                                   name_subsampling_file='phoneme_sub_original_to_20', get_indexes=False)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=6,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_phoneme_20,y=y_phoneme_20,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

In [ ]:
output_dir_path =  "/home/abdoulaye_sakho/S1/th_smote/smote_strategies_study/saved_experiments_2025_v2/not_tuned/res_phoneme/subsample_to_10/" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)
X_phoneme_10,y_phoneme_10 = read_subsampling_indices(X=X_phoneme,y=y_phoneme, dir_subsampling=output_dir_path_lgbm,
                                                   name_subsampling_file='phoneme_sub_20_to_10', get_indexes=False)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=6,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_phoneme_10,y=y_phoneme_10,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

In [ ]:
output_dir_path =  "/home/abdoulaye_sakho/S1/th_smote/smote_strategies_study/saved_experiments_2025_v2/not_tuned/res_phoneme/subsample_to_1/depth_rus" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)
X_phoneme_1,y_phoneme_1 = read_subsampling_indices(X=X_phoneme,y=y_phoneme, dir_subsampling=output_dir_path_lgbm,
                                                   name_subsampling_file='phoneme_sub_10_to_1', get_indexes=False)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=6,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_phoneme_1,y=y_phoneme_1,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> Pima </h1></center>

## Run protocol :

In [ ]:
from data.data import load_pima_data
X_pima, y_pima = load_pima_data()

In [ ]:
smote_bagging = BalancedBaggingClassifier(sampler=SMOTE(),random_state=0)
mgs_bagging = BalancedBaggingClassifier(sampler=MGS(K=9,random_state=0),random_state=0)

In [ ]:
depth_max = None

In [ ]:
lgb_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=8,max_depth=depth_max)
balanced_lgb_model =  RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=8,max_depth=depth_max)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=0)
CVSmotelgb = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=8,max_depth=depth_max),
    list_k_max=15,list_k_step=1)

In [ ]:
output_dir_path =  "/home/abdoulaye_sakho/S1/th_smote/smote_strategies_study/saved_experiments_2025_v2/not_tuned/res_pima/original/" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=9,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    ('SMOTEBagging', NoSampling(), {}, smote_bagging),
    ('MGSBagging', NoSampling(), {}, mgs_bagging),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_pima,y=y_pima,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

In [ ]:
output_dir_path =  "/home/abdoulaye_sakho/S1/th_smote/smote_strategies_study/saved_experiments_2025_v2/not_tuned/res_pima/subsample_to_20/depth_rus" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)
X_pima_20,y_pima_20 = read_subsampling_indices(X=X_pima,y=y_pima, dir_subsampling=output_dir_path_lgbm,
                                               name_subsampling_file='pima_sub_original_to_20', get_indexes=False)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=9,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_pima_20,y=y_pima_20,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> Abalone </h1></center>

## Run protocol :

In [ ]:
from data.data import load_abalone_data
X_abalone, y_abalone = load_abalone_data()

In [ ]:
from collections import Counter
Counter(y_abalone)

In [ ]:
smote_bagging = BalancedBaggingClassifier(sampler=SMOTE(),random_state=0)
mgs_bagging = BalancedBaggingClassifier(sampler=MGS(K=9,random_state=0),random_state=0)

In [ ]:
depth_max=None

In [ ]:
lgb_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=8,max_depth=depth_max)
balanced_lgb_model =  RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=8,max_depth=depth_max)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=0)
CVSmotelgb = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=8,max_depth=depth_max),
    list_k_max=15,list_k_step=1)

In [ ]:
output_dir_path =  "/home/abdoulaye_sakho/S1/th_smote/smote_strategies_study/saved_experiments_2025_v2/not_tuned/res_abalone/original/depth_rus" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=9,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    #('SMOTEBagging', NoSampling(), {}, smote_bagging),
    #('MGSBagging', NoSampling(), {}, mgs_bagging),
    #('SMOTEBoost', NoSampling(), {}, smote_boost),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_abalone,y=y_abalone,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

## Compute metrics :

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> Haberman </h1></center>

## Run protocol :

In [ ]:
from data.data import load_haberman_data
X_haberman,y_haberman = load_haberman_data()

In [ ]:
smote_bagging = BalancedBaggingClassifier(sampler=SMOTE(),random_state=0)
mgs_bagging = BalancedBaggingClassifier(sampler=MGS(K=5,random_state=0),random_state=0)

In [ ]:
depth_max=None

In [ ]:
lgb_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5,max_depth=depth_max)
balanced_lgb_model =  RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5,max_depth=depth_max)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=0)
CVSmotelgb = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5,max_depth=depth_max),
    list_k_max=15,list_k_step=1)

In [ ]:
output_dir_path =  "/home/abdoulaye_sakho/S1/th_smote/smote_strategies_study/saved_experiments_2025_v2/not_tuned/res_haberman/original/" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=5,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    #('SMOTEBagging', NoSampling(), {}, smote_bagging),
    #('MGSBagging', NoSampling(), {}, mgs_bagging),
    #('SMOTEBoost', NoSampling(), {}, smote_boost),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_haberman,y=y_haberman,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

In [ ]:
output_dir_path =  "/home/abdoulaye_sakho/S1/th_smote/smote_strategies_study/saved_experiments_2025_v2/not_tuned/res_haberman/subsample_to_10/depth_rus" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)
X_haberman_10,y_haberman_10 = read_subsampling_indices(X=X_haberman,y=y_haberman, dir_subsampling=output_dir_path_lgbm,
                                                       name_subsampling_file='haberman_sub_original_to_20', get_indexes=False)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=5,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    #('SMOTEBagging', NoSampling(), {}, smote_bagging),
    #('MGSBagging', NoSampling(), {}, mgs_bagging),
    #('SMOTEBoost', NoSampling(), {}, smote_boost),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_haberman_10,y=y_haberman_10,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> Yeast </h1></center>

In [ ]:
from data.data import load_yeast_data
X_yeast,y_yeast = load_yeast_data()

In [ ]:
depth_max=4

In [ ]:
smote_bagging = BalancedBaggingClassifier(sampler=SMOTE(),random_state=0)
mgs_bagging = BalancedBaggingClassifier(sampler=MGS(K=9,random_state=0),random_state=0)

In [ ]:
lgb_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5,max_depth=depth_max)
balanced_lgb_model =  RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5,max_depth=depth_max)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=0)
CVSmotelgb = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5,max_depth=depth_max),
    list_k_max=7,list_k_step=1)

In [ ]:
output_dir_path =  "/home/abdoulaye_sakho/S1/th_smote/smote_strategies_study/saved_experiments_2025_v2/not_tuned/res_yeast/original/" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=9,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    #('SMOTEBagging', NoSampling(), {}, smote_bagging),
    #('MGSBagging', NoSampling(), {}, mgs_bagging),
    #('SMOTEBoost', NoSampling(), {}, smote_boost),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_yeast,y=y_yeast,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

In [ ]:
smote_bagging = BalancedBaggingClassifier(sampler=SMOTE(k_neighbors=3),random_state=0)
mgs_bagging = BalancedBaggingClassifier(sampler=MGS(K=3,random_state=0),random_state=0)

In [ ]:
lgb_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5,max_depth=depth_max)
balanced_lgb_model =  RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5,max_depth=depth_max)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=0)
CVSmotelgb = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5,max_depth=depth_max),
    list_k_max=7,list_k_step=1)

In [ ]:
output_dir_path =  "/home/abdoulaye_sakho/S1/th_smote/smote_strategies_study/saved_experiments_2025_v2/not_tuned/res_yeast/subsample_to_1/depth_rus" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)
X_yeast_1,y_yeast_1 = read_subsampling_indices(X=X_yeast,y=y_yeast, dir_subsampling=output_dir_path_lgbm, name_subsampling_file='yeast_sub_original_to_1', get_indexes=False)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=9,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    #('SMOTEBagging', NoSampling(), {}, smote_bagging),
    #('MGSBagging', NoSampling(), {}, mgs_bagging),
    #('SMOTEBoost', NoSampling(), {}, smote_boost),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_yeast_1,y=y_yeast_1,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> Wine </h1></center>

Wine quality datas et : class 5 and 6 vs 8. Do not forget to install ucimlrepo. # pip install ucimlrepo

In [ ]:
from data.data import load_wine_data
X_wine,y_wine = load_wine_data()

In [ ]:
smote_bagging = BalancedBaggingClassifier(sampler=SMOTE(),random_state=0)
mgs_bagging = BalancedBaggingClassifier(sampler=MGS(K=12,random_state=0),random_state=0)

In [ ]:
depth_max=10

In [ ]:
lgb_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5,max_depth=depth_max)
balanced_lgb_model =  RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5,max_depth=depth_max)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=0)
CVSmotelgb = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5,max_depth=depth_max),
    list_k_max=15,list_k_step=1)

In [ ]:
output_dir_path =  "/home/abdoulaye_sakho/S1/th_smote/smote_strategies_study/saved_experiments_2025_v2/not_tuned/res_wine/original/depth_rus" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=12,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    #('SMOTEBagging', NoSampling(), {}, smote_bagging),
    #('MGSBagging', NoSampling(), {}, mgs_bagging),
    #('SMOTEBoost', NoSampling(), {}, smote_boost),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_wine,y=y_wine,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> Vehicle </h1></center>

## Run protocol :

In [ ]:
from data.data import load_vehicle_data
X_vehicle, y_vehicle = load_vehicle_data()

In [ ]:
indices_kept_10 = subsample_to_ratio_indices(X=X_vehicle,y=y_vehicle,ratio=0.1,seed_sub=11,
    output_dir_subsampling=output_dir_path_lgbm,
    name_subsampling_file='vehicle_sub_original_to_10')

In [ ]:
lgb_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5)
balanced_lgb_model =  RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=0)
CVSmotelgb = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5),
    list_k_max=15,list_k_step=1)

In [ ]:
output_dir_path =  "../saved_experiments_2025_v2/not_tuned/res_bcw/original/res_vehicle/original" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=19,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_vehicle,y=y_vehicle,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

In [ ]:
output_dir_path =  "../saved_experiments_2025_v2/not_tuned/res_bcw/original/res_vehicle/subsample_to_10" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)
X_vehicle_10, y_vehicle_10 = read_subsampling_indices(X=X_vehicle,y=y_vehicle, dir_subsampling=output_dir_path_lgbm, 
                                                      name_subsampling_file='vehicle_sub_original_to_10', get_indexes=False)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=19,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_vehicle_10,y=y_vehicle_10,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> Breast cancer Wisconsin </h1></center>

## Run protocol :

In [ ]:
from data.data import load_breastcancer_data
X_bcw, y_bcw = load_breastcancer_data()

In [ ]:
indices_kept_20 = subsample_to_ratio_indices(X=X_bcw,y=y_bcw,ratio=0.2,seed_sub=11,
                                           output_dir_subsampling=output_dir_path_lgbm,
                                           name_subsampling_file='bcw_sub_original_to_20')

indices_kept_10 = subsample_to_ratio_indices(
    X=X_bcw,y=y_bcw,ratio=0.1,seed_sub=9,
    output_dir_subsampling=output_dir_path_lgbm,
    name_subsampling_file='bcw_sub_20_to_10',has_previous_under_sampling=True,
    previous_under_sampling=indices_kept_20)



In [ ]:
lgb_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5)
balanced_lgb_model =  RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=0)
CVSmotelgb = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5),
    list_k_max=15,list_k_step=1)

In [ ]:
output_dir_path =  "../saved_experiments_2025_v2/not_tuned/res_bcw/original" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=10,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    ]
    
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_bcw,y=y_bcw,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

In [ ]:
output_dir_path =  "../saved_experiments_2025_v2/not_tuned/res_bcw/original/res_bcw/subsample_to_20" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)
X_bcw_20, y_bcw_20 = read_subsampling_indices(X=X_bcw,y=y_bcw, dir_subsampling=output_dir_path_lgbm, 
                                              name_subsampling_file='bcw_sub_original_to_20', get_indexes=False)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=10,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_bcw_20,y=y_bcw_20,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

In [ ]:
output_dir_path =  "../saved_experiments_2025_v2/not_tuned/res_bcw/original/res_bcw/subsample_to_10" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)
X_bcw_10, y_bcw_10 = read_subsampling_indices(X=X_bcw,y=y_bcw, dir_subsampling=output_dir_path_lgbm, 
                                              name_subsampling_file='bcw_sub_20_to_10', get_indexes=False)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=10,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_bcw_10,y=y_bcw_10,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> Ionosphere </h1></center>

## Run protocol :

In [ ]:
from data.data import load_ionosphere_data
X_ionosphere, y_ionosphere = load_ionosphere_data()

In [ ]:
indices_kept_20 = subsample_to_ratio_indices(X=X_ionosphere,y=y_ionosphere,ratio=0.2,seed_sub=11,
                                           output_dir_subsampling=output_dir_path_lgbm,
                                           name_subsampling_file='ionosphere_sub_original_to_20')

indices_kept_10 = subsample_to_ratio_indices(
    X=X_ionosphere,y=y_ionosphere,ratio=0.1,seed_sub=9,
    output_dir_subsampling=output_dir_path_lgbm,
    name_subsampling_file='ionosphere_sub_20_to_10',has_previous_under_sampling=True,
    previous_under_sampling=indices_kept_20)



In [ ]:
lgb_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5)
balanced_lgb_model =  RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=0)
CVSmotelgb = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5),
    list_k_max=15,list_k_step=1)

In [ ]:
output_dir_path =  "../saved_experiments_2025_v2/not_tuned/res_ionosphere/original" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)


init_name_file = '2024-07-08-lgb-tuned'
for i in range(30):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=33,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_ionosphere,y=y_ionosphere,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

In [ ]:
output_dir_path =  "../saved_experiments_2025_v2/not_tuned/res_ionosphere/subsample_to_20" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)
X_ionosphere_20, y_ionosphere_20 = read_subsampling_indices(X=X_ionosphere,y=y_ionosphere, dir_subsampling=output_dir_path_lgbm, 
                                                            name_subsampling_file='ionosphere_sub_original_to_20', get_indexes=False)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(5):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=33,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_ionosphere_20,y=y_ionosphere_20,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

In [ ]:
lgb_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5)
balanced_lgb_model =  RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=0)
CVSmotelgb = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5),
    list_k_max=15,list_k_step=1)

In [ ]:
output_dir_path =  "../saved_experiments_2025_v2/not_tuned/res_ionosphere/subsample_to_10" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)
X_ionosphere_10, y_ionosphere_10 = read_subsampling_indices(X=X_ionosphere,y=y_ionosphere, dir_subsampling=output_dir_path_lgbm, 
                                                            name_subsampling_file='ionosphere_sub_20_to_10', get_indexes=False)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(2):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=15,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_ionosphere_10,y=y_ionosphere_10,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> CreditCard </h1></center>

## Run protocol :

In [ ]:
from data.data import load_credit_data
X_credit, y_credit, meta_df_credit = load_credit_data()

In [ ]:
from validation.classif_experiments import PaperTimeSeriesSplitWithGroupOut
from sklearn.model_selection import TimeSeriesSplit
splitter_PTSSG = PaperTimeSeriesSplitWithGroupOut(n_splits=5, starting_split=0,meta_df=meta_df_credit,col_name_id='Time')

In [ ]:
depth_max=10

In [ ]:
smote_bagging = BalancedBaggingClassifier(sampler=SMOTE(),random_state=0)
mgs_bagging = BalancedBaggingClassifier(sampler=MGS(K=30,random_state=0),random_state=0)

In [ ]:
lgb_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5,max_depth=depth_max)
balanced_lgb_model = RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5,max_depth=depth_max)

splitter_stratified_cv = TimeSeriesSplit(n_splits=2,gap=3)
CVSmotelgb = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5,max_depth=depth_max),
    list_k_max=15,list_k_step=1,take_all_default_value_k=4)

In [ ]:
output_dir_path =  "/home/abdoulaye_sakho/S1/th_smote/smote_strategies_study/saved_experiments_2025_v2/not_tuned/res_credit/original/depth_rus" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(5):
# We do not change the splitter at each iteration here due to the temporal split
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=30,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    ]
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_credit,y=y_credit,
             list_oversampling_and_params=list_oversampling_and_params,to_shuffle=False,
             splitter=splitter_PTSSG)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> MagicTel </h1></center>

## Run protocol : 


In [ ]:
from data.data import load_magictel_data
X_magic,y_magic = load_magictel_data()

In [ ]:
smote_bagging = BalancedBaggingClassifier(sampler=SMOTE(),random_state=0)
mgs_bagging = BalancedBaggingClassifier(sampler=MGS(K=11,random_state=0),random_state=0)

In [ ]:
depth_max=20

In [ ]:
lgb_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5,max_depth=depth_max)
balanced_lgb_model = RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5,max_depth=depth_max)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=0)
CVSmotelgb = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5,max_depth=depth_max),
    list_k_max=15,list_k_step=1)

In [ ]:
X_magic_20,y_magic_20 = read_subsampling_indices(X=X_magic,y=y_magic, dir_subsampling=output_dir_path_lgbm,
                                                 name_subsampling_file='magicTel_sub_original_to_20', get_indexes=False)

In [ ]:
output_dir_path =  "/home/abdoulaye_sakho/S1/th_smote/smote_strategies_study/saved_experiments_2025_v2/not_tuned/res_magic/subsample_to_20/depth_rus" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=11,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    #('SMOTEBagging', NoSampling(), {}, smote_bagging),
    #('MGSBagging', NoSampling(), {}, mgs_bagging),
    #('SMOTEBoost', NoSampling(), {}, smote_boost),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_magic_20,y=y_magic_20,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> California </h1></center>

In [ ]:
from data.data import load_california_data
X_california,y_california = load_california_data()

In [ ]:
smote_bagging = BalancedBaggingClassifier(sampler=SMOTE(),random_state=0)
mgs_bagging = BalancedBaggingClassifier(sampler=MGS(K=9,random_state=0),random_state=0)

In [ ]:
depth_max=None

In [ ]:
lgb_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5,max_depth=depth_max)
balanced_lgb_model = RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5,max_depth=depth_max)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=0)
CVSmotelgb = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5,max_depth=depth_max),
    list_k_max=15,list_k_step=1)

In [ ]:
X_california_1,y_california_1 = read_subsampling_indices(X=X_california,y=y_california, dir_subsampling=output_dir_path_lgbm, 
                                                         name_subsampling_file='california_sub_10_to_1', get_indexes=False)
output_dir_path =  "/home/abdoulaye_sakho/S1/th_smote/smote_strategies_study/saved_experiments_2025_v2/not_tuned/res_california/subsample_to_1" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=9,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    #('SMOTEBagging', NoSampling(), {}, smote_bagging),
    #('MGSBagging', NoSampling(), {}, mgs_bagging),
    #('SMOTEBoost', NoSampling(), {}, smote_boost),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_california_1,y=y_california_1,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

In [ ]:
X_california_20,y_california_20 = read_subsampling_indices(X=X_california,y=y_california, dir_subsampling=output_dir_path_lgbm, 
                                                         name_subsampling_file='california_sub_original_to_20', get_indexes=False)
output_dir_path =  "/home/abdoulaye_sakho/S1/th_smote/smote_strategies_study/saved_experiments_2025_v2/not_tuned/res_california/subsample_to_20/" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=9,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    ('SMOTEBagging', NoSampling(), {}, smote_bagging),
    ('MGSBagging', NoSampling(), {}, mgs_bagging),
    #('SMOTEBoost', NoSampling(), {}, smote_boost),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_california_20,y=y_california_20,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

In [ ]:
X_california_10,y_california_10 = read_subsampling_indices(X=X_california,y=y_california, dir_subsampling=output_dir_path_lgbm, 
                                                         name_subsampling_file='california_sub_20_to_10', get_indexes=False)
output_dir_path =  "/home/abdoulaye_sakho/S1/th_smote/smote_strategies_study/saved_experiments_2025_v2/not_tuned/res_california/subsample_to_10/" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=9,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    ('SMOTEBagging', NoSampling(), {}, smote_bagging),
    ('MGSBagging', NoSampling(), {}, mgs_bagging),
    #('SMOTEBoost', NoSampling(), {}, smote_boost),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_california_10,y=y_california_10,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

<center><h1  style="color:white; background-color:#808b96; border-radius: 10px; padding:15px;"> House_16H </h1></center>

In [ ]:
from data.data import load_house_data
X_house,y_house = load_house_data()

In [ ]:
smote_bagging = BalancedBaggingClassifier(sampler=SMOTE(),random_state=0)
mgs_bagging = BalancedBaggingClassifier(sampler=MGS(K=17,random_state=0),random_state=0)

In [ ]:
lgb_model = RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5)
balanced_lgb_model = RandomForestClassifier(n_estimators=100,criterion='gini',class_weight='balanced',n_jobs=5)

splitter_stratified_cv = StratifiedKFold(n_splits=5,shuffle=True,random_state=0)
CVSmotelgb = CVSmoteModel(
    splitter=splitter_stratified_cv,
    model=RandomForestClassifier(n_estimators=100,criterion='gini',n_jobs=5),
    list_k_max=15,list_k_step=1)

In [ ]:
output_dir_path =  "/home/abdoulaye_sakho/S1/th_smote/smote_strategies_study/saved_experiments_2025_v2/not_tuned/res_house/subsample_to_1/" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)
X_house_1,y_house_1 = read_subsampling_indices(X=X_house,y=y_house, dir_subsampling=output_dir_path_lgbm, 
                                               name_subsampling_file='house16_sub_10_to_1', get_indexes=False)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=17,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    #('SMOTEBagging', NoSampling(), {}, smote_bagging),
    #('MGSBagging', NoSampling(), {}, mgs_bagging),
    #('SMOTEBoost', NoSampling(), {}, smote_boost),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_house_1,y=y_house_1,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

In [ ]:
output_dir_path =  "/home/abdoulaye_sakho/S1/th_smote/smote_strategies_study/saved_experiments_2025_v2/not_tuned/res_house/subsample_to_20/" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)
X_house_20,y_house_20 = read_subsampling_indices(X=X_house,y=y_house, dir_subsampling=output_dir_path_lgbm, 
                                               name_subsampling_file='house16_sub_original_to_20', get_indexes=False)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(20):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=17,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    #('SMOTEBagging', NoSampling(), {}, smote_bagging),
    #('MGSBagging', NoSampling(), {}, mgs_bagging),
    #('SMOTEBoost', NoSampling(), {}, smote_boost),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_house_20,y=y_house_20,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

In [ ]:
output_dir_path =  "/home/abdoulaye_sakho/S1/th_smote/smote_strategies_study/saved_experiments_2025_v2/not_tuned/res_house/subsample_to_10/" 
Path(output_dir_path).mkdir(parents=True, exist_ok=True)
X_house_10,y_house_10 = read_subsampling_indices(X=X_house,y=y_house, dir_subsampling=output_dir_path_lgbm, 
                                               name_subsampling_file='house16_sub_20_to_10', get_indexes=False)

init_name_file = '2024-07-08-lgb-tuned'
for i in range(3):
    list_oversampling_and_params = [
    ('None', NoSampling(), {}, lgb_model),
    ('c_weight', NoSampling(), {}, balanced_lgb_model),
    ('RUS', RandomUnderSampler(sampling_strategy="majority",replacement=False,random_state=i),{}, lgb_model),
    ('ROS',  RandomOverSampler(sampling_strategy="minority",random_state=i),{}, lgb_model),
    ('NearMiss1', NearMiss(sampling_strategy="majority", version=1),{}, lgb_model),
    ('BorderlineS_1', BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-1",random_state=i),{}, lgb_model),
    ('BorderlineS_2',BorderlineSMOTE(sampling_strategy="minority", k_neighbors=5, kind="borderline-2",random_state=i),{}, lgb_model),
    ('cv_smote', NoSampling(),{}, CVSmotelgb),
    ('SMOTE',SMOTE(sampling_strategy="minority",random_state=i),{},lgb_model),
    ('MGS',MGS(K=17,random_state=i),{},lgb_model),
    ('CTGAN', CtganSampler(epochs=10,generator_lr=2e-4,discriminator_lr=2e-4,random_state=i), {}, lgb_model),
    ('ForestDiff', ForesDiffSampler(random_state=i), {}, lgb_model),
    #('SMOTEBagging', NoSampling(), {}, smote_bagging),
    #('MGSBagging', NoSampling(), {}, mgs_bagging),
    #('SMOTEBoost', NoSampling(), {}, smote_boost),
    ]
    splitter_stratified = StratifiedKFold(n_splits=5,shuffle=True,random_state=100+i)
    name_file = init_name_file + str(i) +'.npy'
    run_eval(output_dir=output_dir_path,name_file=name_file,X=X_house_10,y=y_house_10,
             list_oversampling_and_params=list_oversampling_and_params,
             splitter=splitter_stratified)

<center><h1  style="color:white; background-color:#008b96; border-radius: 10px; padding:15px;"> END </h1></center>